In [2]:
# 배달 속도 1~ 3시간 사이
# 가격도 비싸진 않다.
# 마트 상품을 배달
# 식재료, 신선제품, 택배로 못받을것을...
# 미싱 아더 제외
# 홈파티를 위해 재료를 요일데이터 비교 알콜이 몇등일까?
# 어떤구매자가 리오더를 했을까?
# 구매했던애랑 안했던애랑 차이점

In [21]:
pd.set_option('display.max_row', 300)
pd.set_option('display.max_column', 100)

In [4]:
raw_products = pd.read_csv('data/products.csv')
raw_products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [5]:
raw_aisles = pd.read_csv('data/aisles.csv')
raw_aisles.tail()

,aisle_id,aisle
129,130,hot cereal pancake mixes
130,131,dry pasta
131,132,beauty
132,133,muscles joints pain relief
133,134,specialty wines champagnes


In [6]:
raw_departments = pd.read_csv('data/departments.csv')
raw_departments.tail()

,department_id,department
16,17,household
17,18,babies
18,19,snacks
19,20,deli
20,21,missing


In [7]:
raw_orders = pd.read_csv('data/orders.csv')
raw_orders.tail()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0
3421082,272231,206209,train,14,6,14,30.0


In [8]:
raw_order_products_train = pd.read_csv('data/order_products__train.csv')
raw_order_products = pd.read_csv('data/order_products.csv')
raw_datas = pd.concat([raw_order_products, raw_order_products_train])
raw_datas.sort_values(by='order_id', inplace=True)
raw_datas.reset_index(drop=True, inplace=True)
raw_datas.tail()

,order_id,product_id,add_to_cart_order,reordered
33819101,3421083,35211,5,0
33819102,3421083,39678,6,1
33819103,3421083,11352,7,0
33819104,3421083,4600,8,0
33819105,3421083,24852,9,1


- product_id: 제품 식별자
- product_name: 제품 이름
- aisle_id외래 키
- department_id외래 키
- order_id: 주문 식별자
- user_id: 고객 식별자
- eval_set:이 순서가 속하는 평가 세트 (아래 SET설명 참조)
- order_number:이 사용자의 주문 순서 번호 (1 = 첫 번째, n = n 번째)
- order_dow: 주문한 요일
- order_hour_of_day: 주문한 시간
- days_since_prior: 마지막 주문 이후 일 수, 30 일 제한 (NA는 order_number1) 재주문 기간의 텀
- order_id외래 키
- product_id외래 키
- add_to_cart_order: 각 제품이 장바구니에 추가 된 순서
- reordered: 과거에이 사용자가이 제품을 주문한 경우 1, 그렇지 않으면 0
- department_id: 부서 식별자
- department: 부서명
- aisle_id: aisles_id
- aisle: name

In [9]:
# 데이터 merge
raw_data = pd.merge(raw_products, raw_aisles)
raw_data = pd.merge(raw_data, raw_departments)
raw_data = pd.merge(raw_data, raw_datas)
raw_data = pd.merge(raw_data, raw_orders)
# 33819106 x 15
raw_data.tail()

,product_id,product_name,aisle_id,department_id,aisle,department,order_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
33819101,30365,Vegetable Chips,18,10,bulk dried fruits vegetables,bulk,2323735,1,1,90235,prior,30,3,10,1.0
33819102,30365,Vegetable Chips,18,10,bulk dried fruits vegetables,bulk,2733396,1,0,30511,prior,24,4,11,0.0
33819103,30365,Vegetable Chips,18,10,bulk dried fruits vegetables,bulk,3152383,1,0,10113,train,12,3,19,13.0
33819104,48778,"Fit Super A Juice, Cold Pressed, Carrot/Apple/...",18,10,bulk dried fruits vegetables,bulk,31522,1,1,31188,prior,3,3,11,4.0
33819105,48778,"Fit Super A Juice, Cold Pressed, Carrot/Apple/...",18,10,bulk dried fruits vegetables,bulk,2484167,1,1,31188,prior,7,2,11,10.0


### aisle 별 재 구매율 비율

In [10]:
# 분석용 데이터 프레임 생성
columns = ['aisle_id', 'reordered', 'aisle']
data_aisle = raw_data[columns]
data_aisle

,aisle_id,reordered,aisle
0,61,0,cookies cakes
1,77,0,soft drinks
2,34,0,frozen meat seafood
3,21,0,packaged cheese
4,21,0,packaged cheese
...,...,...,...
33819101,18,1,bulk dried fruits vegetables
33819102,18,0,bulk dried fruits vegetables
33819103,18,0,bulk dried fruits vegetables
33819104,18,1,bulk dried fruits vegetables


In [11]:
# 피봇테이블 생성
aisle_table = data_aisle.pivot_table(index='aisle', columns='reordered', aggfunc='count')
columns = ['first_order','reorder']
aisle_table.columns = columns

In [12]:
# 피봇테이블 정리
aisle_table['aisle_total'] = aisle_table['first_order'] + aisle_table['reorder']
aisle_table['aisle_reorder_rate'] =  aisle_table['reorder'] / aisle_table['aisle_total'] * 100
aisle_table['aisle_of_reorder'] = aisle_table['reorder'].sum() / aisle_table['aisle_total'].sum() * 100
aisle_table.sort_values(by = 'aisle_reorder_rate', ascending = False, inplace=True)
aisle_table.head()

,first_order,reorder,aisle_total,aisle_reorder_rate,aisle_of_reorder
aisle,,,,,
milk,201531,722128,923659,78.181234,59.006172
water seltzer sparkling water,237162,640988,878150,72.992997,59.006172
fresh fruits,1066410,2726251,3792661,71.882275,59.006172
eggs,138601,333408,472009,70.635941,59.006172
soy lactosefree,204424,460069,664493,69.236094,59.006172


### product 별 재구매율

In [13]:
# 분석용 데이터 생성
columns = ['product_id', 'reordered', 'product_name']
data_product = raw_data[columns]
data_product.head()

,product_id,reordered,product_name
0,1,0,Chocolate Sandwich Cookies
1,46149,0,Zero Calorie Cola
2,9434,0,Bag of Large Lemons
3,769,0,Sliced American Cheese
4,12078,0,Shredded Mexican Blend Cheese


In [14]:
# 피봇테이블 생성
product_table = data_product.pivot_table(index='product_name', columns='reordered', aggfunc='count', fill_value = 0)
columns = ['first_order','reorder']
product_table.columns = columns
product_table

,first_order,reorder
product_name,,
#2 Coffee Filters,534,265
#2 Cone White Coffee Filters,11,3
#2 Mechanical Pencils,9,0
#4 Natural Brown Coffee Filters,157,90
& Go! Hazelnut Spread + Pretzel Sticks,81,93
...,...,...
with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,24,17
with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum,37,20
with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,40,34


In [15]:
# 테이블 정리
product_table['product_total'] = product_table['first_order'] + product_table['reorder']
product_table['product_reorder_rate'] =  product_table['reorder'] / product_table['product_total'] * 100
product_table['total_of_reorder'] = product_table['reorder'].sum() / product_table['product_total'].sum() * 100
product_table.sort_values(by = 'product_reorder_rate', ascending = False, inplace=True)
product_table.head()

,first_order,reorder,product_total,product_reorder_rate,total_of_reorder
product_name,,,,,
Raw Veggie Wrappers,4,65,69,94.202899,59.006172
Serenity Ultimate Extrema Overnight Pads,6,84,90,93.333333,59.006172
Orange Energy Shots,1,12,13,92.307692,59.006172
Chocolate Love Bar,8,94,102,92.156863,59.006172
Soy Powder Infant Formula,3,32,35,91.428571,59.006172


In [16]:
# 테이블의 함수화
def numoforder(n):
     return product_table[product_table['product_total'] >= n]

In [23]:
# 몇개 이상의 제품이 팔려야 많이 팔린것인가?
n = 1000
print(numoforder(n)['product_total'].sum())
print(product_table['product_total'].sum())
print(f'{n}개 팔리면 전체의',round(numoforder(n)['product_total'].sum() / product_table['product_total'].sum() * 100,2),'% 입니다' )

27793456
33819106
1000개 팔리면 전체의 82.18 % 입니다


#### 시간, 주별 reorder

In [18]:
# 시간,주 별 데이터 생성
columns = [	'reordered', 'order_dow', 'order_hour_of_day']
data_time = raw_data[columns]
data_time.head()

,reordered,order_dow,order_hour_of_day
0,0,1,11
1,0,1,11
2,0,1,11
3,0,1,11
4,0,1,11


In [19]:
# reorder 테이블 정리
time_table = data_time.pivot_table(
    index='order_dow', columns='order_hour_of_day', 
    values='reordered', aggfunc=sum)
time_table

order_hour_of_day,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
order_dow,,,,,,,,,,,,,,,,,,,,,,,,
0,23318,13088,7527,5330,4796,7271,22719,90051,202307,281462,320637,326554,324176,339622,343522,336483,304386,238533,172145,133623,115072,98382,76120,44776
1,22759,11165,6198,4207,4730,9362,36182,114678,227467,339434,356050,320117,284224,279945,278150,267876,252249,203956,155109,118246,90926,72658,54793,34180
2,17498,8545,4913,4252,4046,8102,28896,81695,146262,211211,224302,215118,198475,201651,202992,201412,200205,166025,135702,102354,77343,62000,47764,31243
3,15800,7607,4951,3349,3952,7420,26842,73910,128257,176628,193230,189174,177730,182395,181435,189730,184054,157514,128052,98622,73614,60623,49531,29857
4,14335,7713,4601,3702,4196,7631,26654,75208,125848,170766,188926,183645,174639,178642,180787,182992,181256,155605,126362,102484,80411,67168,54619,32430
5,16958,8846,5550,4761,5467,9479,30382,85071,144918,202414,223285,218823,203663,206354,214828,213053,203166,166158,131138,99925,76840,61526,48667,32616
6,18335,10802,6628,4531,4702,6668,21123,77794,155115,198390,220069,222840,220991,223512,233574,223637,207837,170587,135483,102283,79117,69207,56496,38287


In [20]:
# 전체 테이블 정리
time_table_total = data_time.pivot_table(
    index='order_dow', columns='order_hour_of_day', 
    values='reordered', aggfunc=len)
time_table_total

order_hour_of_day,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
order_dow,,,,,,,,,,,,,,,,,,,,,,,,
0,40444,23647,13566,9375,8739,12242,36425,140403,321927,462336,542965,564267,563020,587135,595896,581287,527908,416816,300095,229047,192892,161558,126891,74811
1,38560,18866,11132,7305,7919,14979,54881,170880,343737,521484,568411,526566,479604,475107,471210,459764,435568,352051,270431,206866,157753,124681,94913,59166
2,30957,15250,8693,7352,7144,13243,45158,127152,231210,339918,371872,365260,341984,347977,348250,347439,343865,289659,236761,178193,136254,106898,83471,54400
3,28513,14205,9111,6093,7196,12253,42426,115125,206371,288048,325491,323729,308309,314999,316153,326788,318655,272577,222248,171472,127118,104338,84872,52408
4,25740,14281,8434,6596,7182,12560,41609,118105,201808,280837,317797,313125,301061,305625,309373,312141,309872,264970,216530,176132,135829,112755,93557,56777
5,30890,16057,9894,8704,9232,15368,48189,134439,232906,327762,369283,368698,348700,350678,360662,357337,340423,282285,227529,174797,130929,103320,82167,56194
6,32927,19106,11830,8334,8302,11264,33954,122135,249400,330184,379086,391056,389921,396533,409374,395975,371404,308051,240851,181069,137183,117633,96182,65829
